# EASYMORE Basin Subset
EASYMORE: EArth SYstem MOdeling REmapper is a collection of functions that allows extraction of the data from a NetCDF file for a given shapefile such as a basin, catchment, points or lines. It can map gridded data or model output to any given shapefile and provide area average for a target variable.
https://github.com/ShervanGharari/EASYMORE

## Climate Forcing Remapping
This script will map the gridded forcing data, specified in control_active.txt to the subasins contained within the basin shapefile used to produce the drainage database.

### Loading Modules

In [1]:
from easymore.easymore import easymore
from pathlib import Path
from shutil import copyfile
from datetime import datetime

### Control File Handling
The purpose of the control file is to provide all inputs to the scripts in the vector-based workflow to eliminate the need to alter the workflow scripts themselves. The following cells will retrieve settings from 'control_active.txt' and provide them as inputs to this script.

##### Access to the control file folder

In [2]:
controlFolder = Path('../0_control_files')

##### Store the name of the 'active' file in a variable

In [3]:
controlFile = 'control_active.txt'

##### Function to extract a given setting from the control file

In [4]:
def read_from_control( file, setting ):
     
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
             
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
     
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
        
    # Return this value   
    return substring

##### Function to specify a default path

In [5]:
def make_default_path(suffix):
     
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
     
    # Specify the forcing path
    #defaultPath = rootPath / domainFolder / suffix
    defaultPath = rootPath / suffix 
    return defaultPath

##### Get the domain folder

In [6]:
domain_name = read_from_control(controlFolder/controlFile,'domain_name')
domainFolder = 'domain_' + domain_name

##### Get the target shapefile

In [7]:
target_shp_path = read_from_control(controlFolder/controlFile,'river_basin_shp_path')
# Specify default path if needed
if target_shp_path == 'default':
    target_shp_path = make_default_path('shapefiles/catchment/') # outputs a Path()
else:
    target_shp_path = Path(target_shp_path) # make sure a user-specified path is a Path()
target_shp_name = read_from_control(controlFolder/controlFile,'river_basin_shp_name')
target_shp      = target_shp_path / target_shp_name
target_shp_ID   = read_from_control(controlFolder/controlFile,'river_basin_shp_rm_hruid')

##### Get the source forcing file

In [8]:
source_nc_path  = read_from_control(controlFolder/controlFile,'source_nc_path')
# Specify default path if needed
if source_nc_path == 'default':
    source_nc_path = make_default_path('forcing') # outputs a Path()
else:
    source_nc_path = Path(source_nc_path) # make sure a user-specified path is a Path()
source_nc_name  = read_from_control(controlFolder/controlFile,'source_nc_name')
source_nc     = str(source_nc_path / source_nc_name)

##### Get the output location

In [9]:
outdir = read_from_control(controlFolder/controlFile,'remapping_out')
if outdir == 'default':
    outdir = str(make_default_path('forcing/'))+'/' # outputs a Path()
else:
    outdir = outdir # make sure a user-specified path is a Path()

##### Get the list of variable names

In [10]:
var_names       = read_from_control(controlFolder/controlFile,'var_names').split(', ')

##### Get the forcing dataset name

In [11]:
forcing_dataset = read_from_control(controlFolder/controlFile,'forcing_dataset')

### Starting EASYMORE

##### Initializing EASYMORE object

In [12]:
esmr = easymore()

EASYMORE version 0.0.4 is initiated.


##### Specifying EASYMORE objects

In [13]:
# name of the case; the temporary, remapping and remapped file names include case name
esmr.case_name                = '{}_{}'.format(forcing_dataset,domain_name)
# temporary path that the EASYMORE generated GIS files and remapped file will be saved
esmr.temp_dir                 = '{}/temporary{}/'.format(source_nc_path,domain_name)
# name of target shapefile that the source netcdf files should be remapped to
esmr.target_shp               = target_shp
esmr.target_shp_ID            = target_shp_ID
# name of netCDF file(s); multiple files can be specified with *
esmr.source_nc                = source_nc 
# name of variables from source netCDF file(s) to be remapped
esmr.var_names                = var_names
# rename the variables from source netCDF file(s) in the remapped files;
# it will be the same as source if not provided
#esmr.var_names_remapped       = ['RDRS']
# name of variable longitude in source netCDF files
esmr.var_lon                  =  read_from_control(controlFolder/controlFile,'var_lon')
# name of variable latitude in source netCDF files
esmr.var_lat                  = read_from_control(controlFolder/controlFile,'var_lat')
# name of variable time in source netCDF file; should be always time
esmr.var_time                 = read_from_control(controlFolder/controlFile,'var_time')
# location where the remapped netCDF file will be saved
esmr.output_dir               = outdir
# format of the variables to be saved in remapped files,
# if one format provided it will be expanded to other variables
esmr.format_list              = read_from_control(controlFolder/controlFile,'format_list').split(', ')
# fill values of the variables to be saved in remapped files,
# if one value provided it will be expanded to other variables
esmr.fill_value_list          = read_from_control(controlFolder/controlFile,'fill_value_list').split(', ')


# if required that the remapped values to be saved as csv as well
#esmr.save_csv                 = True
#esmr.complevel                 =  9
# if uncommented EASYMORE will use this and skip GIS tasks
#esmr.remap_csv                = 'temporary78/subbasin_select/RDRS_78_remapping.csv' # RDRS_81_remapping.csv


##### Create source shapefile

In [14]:
import geopandas as gpd
esmr.NetCDF_SHP_lat_lon()
# create the source shapefile for case 1 and 2 if shapefile is not provided
if (esmr.case == 1 or esmr.case == 2)  and (esmr.source_shp == ''):
    if esmr.case == 1:
        if hasattr(esmr, 'lat_expanded') and hasattr(esmr, 'lon_expanded'):
            esmr.lat_lon_SHP(esmr.lat_expanded, esmr.lon_expanded,\
                esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
        else:
            esmr.lat_lon_SHP(esmr.lat, esmr.lon,\
                esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
    else:
        esmr.lat_lon_SHP(esmr.lat, esmr.lon,\
            esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
    print('EASYMORE is creating the shapefile from the netCDF file and saving it here:')
    print(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')

shp = gpd.read_file(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')
shp = shp [shp['lon_s']>-179]
shp.to_file(esmr.temp_dir+esmr.case_name+'_source_shapefile.shp')

# add the source shapefile 
esmr.source_shp                =   esmr.temp_dir+esmr.case_name+'_source_shapefile.shp'
esmr.source_shp_lat            =  'lat_s' # name of column latitude in the source shapefile
esmr.source_shp_lon            =  'lon_s' # name of column longitude in the source shapefile
esmr.source_shp_ID             =  'ID_s' # name of column ID in the source shapefile

EASYMORE detects case 2 - rotated lat/lon
EASYMORE is creating the shapefile from the netCDF file and saving it here:
C:\Users\5600x\Desktop\GWF\MESH-Scripts\Model_Workflow\forcing/temporaryBowAtBanff/RDRS_BowAtBanff_source_shapefile.shp


c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


### Execute EASYMORE

In [15]:
esmr.nc_remapper()

no author name is provide and the author name is changed to (author name)!
EASYMORE is given multiple varibales to be remapped but only on format and fill valueEASYMORE repeat the format and fill value for all the variables in output files
EASYMORE will remap variable  RDRS_v2.1_A_PR0_SFC  from source file to variable  RDRS_v2.1_A_PR0_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_FI_SFC  from source file to variable  RDRS_v2.1_P_FI_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_FB_SFC  from source file to variable  RDRS_v2.1_P_FB_SFC  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_TT_09944  from source file to variable  RDRS_v2.1_P_TT_09944  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_UVC_09944  from source file to variable  RDRS_v2.1_P_UVC_09944  in remapped NeCDF file
EASYMORE will remap variable  RDRS_v2.1_P_P0_SFC  from source file to variable  RDRS_v2.1_P_P0_SFC  in remapped NeCDF file
EASYMORE w

c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:323: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:323: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:


EASYMORE detect the shapefile is provided and will resave it here:
C:\Users\5600x\Desktop\GWF\MESH-Scripts\Model_Workflow\forcing/temporaryBowAtBanff/RDRS_BowAtBanff_source_shapefile.shp
EASYMORE detects that shapefile longitude is between -180 and 180, no correction is performed
inside shp_lon_correction, no crs is provided for the shapefile; EASYMORE will allocate WGS84 to correct for lon above 180
EASYMORE detects that shapefile longitude is between -180 and 180, no correction is performed


c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:323: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
c:\Users\5600x\anaconda3\envs\easymore\lib\site-packages\geopandas\io\file.py:323: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:
c:\Users\5

------REMAPPING------
netcdf output file will be compressed at level 4
Remapping C:\Users\5600x\Desktop\GWF\MESH-Scripts\Model_Workflow\forcing\rdrsv2.1_1980-01-0107.nc to C:\Users\5600x\Desktop\GWF\MESH-Scripts\Model_Workflow\forcing/RDRS_BowAtBanff_remapped_1980-01-01-13-00-00.nc
Started at date and time 2023-01-26 15:14:57.901033
Ended   at date and time 2023-01-26 15:15:00.953962
------


### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.


In [16]:
# Set the log path and file name
logPath = Path(outdir)
log_suffix = '_easymore_remapping.txt'
 
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)
 
# Copy this script
thisFile = '2_easymore_remapping.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);
 
# Get current date and time
now = datetime.now()
 
# Create a log file
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
     
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated remapped climate forcing .nc file.']
    for txt in lines:
        file.write(txt)